# Exercise1

## Starting Point

We start osberving the data undertsanding that th einformation we need come from the 7th line till the end.

In [2]:
n = 0
with open("USA-road-d.CAL.gr", encoding='utf-8') as file:
    for line in file:
        if (n<10):
            print(line)
            n += 1

c 9th DIMACS Implementation Challenge: Shortest Paths

c http://www.dis.uniroma1.it/~challenge9

c TIGER/Line graph USA-road-d.CAL

c

p sp 1890815 4657742

c graph contains 1890815 nodes and 4657742 arcs

c

a 1 1048577 456

a 1048577 1 456

a 2 1048578 2389



In [3]:
n = 0
with open("USA-road-t.CAL.gr", encoding='utf-8') as file:
    for line in file:
        if (n<10):
            print(line)
            n += 1

c 9th DIMACS Implementation Challenge: Shortest Paths

c http://www.dis.uniroma1.it/~challenge9

c TIGER/Line graph USA-road-t.CAL

c

p sp 1890815 4657742

c graph contains 1890815 nodes and 4657742 arcs

c

a 1 1048577 1139

a 1048577 1 1139

a 2 1048578 5972



Let's see the structure. It seems that splitting the lines we have easily the informations we need and this will work for all three files. So let's start!

In [4]:
n = 0
with open("USA-road-d.CAL.co", encoding='utf-8') as file:
    for line in file:
        if (n>6 and n<16):
            ww = line.split()
            print(line)
            print(ww)
        n += 1

v 1 -114315309 34133550

['v', '1', '-114315309', '34133550']
v 2 -114223946 34176221

['v', '2', '-114223946', '34176221']
v 3 -114307299 34148791

['v', '3', '-114307299', '34148791']
v 4 -114318765 34138889

['v', '4', '-114318765', '34138889']
v 5 -114347300 34042614

['v', '5', '-114347300', '34042614']
v 6 -114347162 34058931

['v', '6', '-114347162', '34058931']
v 7 -114126950 33988170

['v', '7', '-114126950', '33988170']
v 8 -114347268 34028261

['v', '8', '-114347268', '34028261']
v 9 -114340613 34042519

['v', '9', '-114340613', '34042519']


So the structure is pretty clear, we start in row 7 and listing them we can access to the info explained in the exercise text.

Now We start creating ex2, where we will just continue reading these 3 files and in the meantime we will save the info as nodes, arcs, latitudes and longitudes, distances d and distances t in our big Graph structure.

# Exercise 2 

In [28]:
from collections import defaultdict
import itertools
import random
import sys

## The Class Graph

We created a Class Graph that had in itself.<br> A dictionary with key=node, values=nodes adjacents to the node (self.adjacency).<br>A dictionary of coordinates with key=node, value=[lat, long] (self.coordinates).<br>A dict of lists with key=node, values=list of adjiancent nodes i and distances (i,node) in the form node:[[i,d_inode],...] (self.adj_list_d,t or n in base of the kind of distance utilized). <br>
To make all this we used object orientd programming united with functional programming(the "kind" for example you find the the dijsktrafor2,3,4).

## What did we do?<br>
For functionality1 we just performed a bfs, in regard at the cited distance.<br>
For functionality2 we had in mind a torusit that want to visit all monuments with less effort, so we did all permutation of the nodes we want to touch, analyzed the paths with the dijsktra algo(for the minimum path between two nodes) and choosed the best path.<br>
For functionality3 we used dijsktra in order for every two nodes given and print out the path as edges.<br>
For functionality4 we did a little variations of functionality2 with this time a fiex starting point and end point, but the idea is the same. When the user gives more than 7 nodes, we willapply and euristic approach, sampling from the combinations of nodes (except start and end) and assessing in 450 tries a local minimum. This method could certantly be wrong, but for sure will give a local minimum and will not overload the memory.

In [136]:
from collections import defaultdict
class Graph(object):
    '''
    Graph Data Structure, undirected 
    '''
    
    def __init__(self):
        self.adjacency = defaultdict(set)
        self.coordinates = {}
        self.adj_list_d = defaultdict(list)
        self.adj_list_t = defaultdict(list)
        self.adj_list_n = defaultdict(list)
        
    def add(self, node1, node2):
        ''' Add connection between node1, node2'''
        self.adjacency[int(node1)].add(int(node2))
        self.adjacency[int(node2)].add(int(node1))
        #self.net[int(node1), int(node2)] = 1
    
    def distance(self, node1, node2, d):
        ''' Create the distance measure between node1 and node2 '''
        #self.dis[(int(node1), int(node2))] = int(d)
        if [int(d), int(node2)] not in self.adj_list_d[int(node1)]:
            self.adj_list_d[int(node1)].append([int(d), int(node2)])
            self.adj_list_d[int(node2)].append([int(d), int(node1)])
            self.adj_list_n[int(node1)].append([1, int(node2)])
            self.adj_list_n[int(node2)].append([1, int(node1)])
        
    def time(self, node1, node2, t):
        ''' Create the time distance measure between node 1 and node 2'''
        
        #self.tim[(int(node1), int(node2))] = int(t)
        if [ int(t), int(node2)] not in  self.adj_list_t[int(node1)]:
            self.adj_list_t[int(node1)].append([ int(t), int(node2)])
            self.adj_list_t[int(node2)].append([int(t), int(node1)])
        
    def coordinate(self, node, coordinate1, coordinate2):
        '''Save the coordinates of every node'''
        self.coordinates[int(node)] = [int(coordinate1), int(coordinate2)]
    
    def nodes_(self):
        ''' All the nodes of the graph '''
        return list(set(self.adjacency.keys()))
    
    def edges(self):
        ''' Return all the edges of the graph '''
        return list(self.dis.keys())
    
    def print_adj_list_d(self):
        for keys, values in self.adj_list_d.items():
            print(keys, values)
            
    def print_adj_list_t(self):
        for keys, values in self.adj_list_d.items():
            print( keys, values)
    ''' We will use the dfs to make a list of nodes where  
             we can start my MST algorithm in functionality2
             because running all the tree we can find the connected component with one node
     '''   
    def dfs(self, startnode, visited, kind): #kind is a function that will be used to recall self.adj_list_d,t or n in base of the user request
        if visited is None:
            visited = set() 
        visited.add(startnode)
        for i in kind[startnode]:
            if i[1] not in visited:
                    self.dfs(i[1], visited, kind)
        return visited
    '''We use BFS because it returns the minimal graph from a starting point, and because
    we care about <=distance we modified the algo to work with the weights'''
    def bfs(self, startnode, dist, kind):
        visited = []
        queue = [startnode]
        distqueue = [0]
        while queue:
            node = queue.pop(0)
            dqueue = distqueue.pop(0)
            if dqueue <= dist and node not in visited:
                visited.append(node)
                if dqueue <= dist:
                    for neigh in kind[node]:
                        queue.append(neigh[1])
                        distqueue.append(neigh[0]+dqueue)
        return visited
            
    def functionality1(self):
        startnode = int(input("Give me the node as a number: \n "))
        measure = int(input("We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. \n "))
        dist = int(input("Put a number as threshold distance: \n "))
        if measure == 1:
            return (self.bfs(startnode, dist, self.adj_list_d), measure)
        elif measure == 2:
            return (self.bfs(startnode, dist, self.adj_list_t), measure)
        elif measure == 3 :
            return (self.bfs(startnode, dist, self.adj_list_n),measure)
        
    def mst(self, nodes, kind):
        n = len(nodes)
        originals = nodes
        tree = []
        nodesfixed = []
        minweight = sys.float_info.max
        tempath = []
        for i in range(n):
                for j in range(i+1, n):
                    A, B = self.dijsktra(nodes[i], nodes[j], kind)
                    if B != []:
                        #print(B)
                        if B < minweight:
                            minweight = B
                            temporary = [nodes[i],nodes[j]]
                            tempath = A
        if minweight == sys.float_info.max:
            return('NO PATH POSSIBLE; TOTALLY DISCONNECTED NODES')
        for h in tempath:
            tree.append(h)
        #tree.append(h for h in tempath)
        for x in temporary:
            nodesfixed.append(x)
        #nodesfixed.append(x for x in temporary)
        nn = []
        for no in nodes:
            if no not in temporary:
                nn.append(no)
        nodes = nn
        if n>2:
            while(nodes != []):
                minweight = sys.float_info.max
                tempath = []
                for i in nodesfixed:
                    for j in nodes:
                        A, B = self.dijsktra(i, j, kind) 
                        if B != []:
                            if B < minweight:
                                minweight = B
                                temporary = [j]
                                tempath = A
                if minweight ==  sys.float_info.max:
                    print("The nodes are not all connected, this is the tree we could achieve, and the nodes disconnected from the one we took as starting points")
                    return list(set(tree))
                for h in tempath:
                    tree.append(h)
                for x in temporary:
                    nodesfixed.append(x)
                nn = []
                for no in nodes:
                    if no not in temporary:
                        nn.append(no)
                nodes = nn
            return list(set(tree))
                
    def functionality2(self):
        startnodes = [int(x) for x in input('Write the nodes, each one spaced from the other.').split()]
        measure = int(input("We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. \n "))
       # connectedcomp = self.dfs(startnodes[0], None, self.adj_list_n)
      #  rdn = random.choiche(connectedcomp)
        if measure == 1:
            return (self.mst(startnodes, self.adj_list_d), startnodes, measure)
        elif measure == 2:
            return (self.mst(startnodes,self.adj_list_t), startnodes, measure)
        elif measure == 3 :
            return (self.mst(startnodes,self.adj_list_n), startnodes, measure)
    
    '''This Dijsktra return a list of nodes that you will touch in the minimum path.
    The code works exactly like the dijsktra method renewing each time the weight and predecessor of a node. 
    We point the attention that the weights arer always counted for all the path, so every node
    in shortest_paths will have a weight assocviated to a path, not only a couple of nodes'''
    def dijsktra(self, initial, end, kind):
    # shortest paths is a dict of nodes
    # whose value is a tuple of (previous node, weight)
        shortest_paths = {initial: (None, 0)}
        current_node = initial
        visited = set()
    
        while current_node != end:
            visited.add(current_node)
            destinations = kind[current_node]
            weight_to_current_node = shortest_paths[current_node][1]

            for next_node in destinations:
                weight = next_node[0] + weight_to_current_node
                if next_node[1] not in shortest_paths:
                    shortest_paths[next_node[1]] = (current_node, weight)
                else:
                    current_shortest_weight = shortest_paths[next_node[1]][1]
                    if current_shortest_weight > weight:
                        shortest_paths[next_node[1]] = (current_node, weight)
            next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
            if not next_destinations:
               # return "Route Not Possible" as a [] that later on will be checked if it remains so
                return [],[]
        # next node is the destination with the lowest weight in the second component
            current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
           
    
    # Work back through destinations in shortest path
        path = []
        while current_node is not None:
            path.append(current_node)
            next_node = shortest_paths[current_node][0]
            current_node = next_node
        
        path = path[::-1]
        tup = []
        for i in range(len(path)-1):
            tup.append((path[i], path[i+1]))
        return tup, shortest_paths[end][1]
    
    def dijsktrafor3(self, startnode, nodes, kind):
        nodi = [startnode]
        for i in nodes:
            nodi.append(i)
        final_path = []
        final_weight = 0
        to_print = []
        for i in range(len(nodi)-1):
            A, B = self.dijsktra(nodi[i], nodi[i+1], kind)
            if A == []:
                    return 'No path ordered allowed'
            final_weight += B
            final_path.append(A)
        for i in final_path:
            for j in i:
                to_print.append(j)
        return to_print
    
    def functionality3(self):
        H_n = int(input('Write the number associated to the starting node.\n'))
        startnodes = [int(x) for x in input('Write the nodes, each one spaced from the other.\n').split()]
        measure = int(input("We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. \n "))
        if measure == 1:
            return (self.dijsktrafor3( H_n, startnodes, self.adj_list_d), H_n, startnodes, measure)
        elif measure == 2:
            return (self.dijsktrafor3( H_n, startnodes, self.adj_list_t), H_n, startnodes, measure)
        elif measure == 3 :
            return (self.dijsktrafor3( H_n, startnodes, self.adj_list_n), H_n, startnodes, measure)
        
    def dijsktrafor4(self, startnode, nodes, endnode, kind):  
        n = len(nodes)
        final_path = []
        weight_final = sys.float_info.max 
        if n > 5:
            set.seed(123)
            print('The nodes are more than 7, so we use an euristic approach based on random samples, that will give an approximate local minimum for our datasets')
            for j in range(450):
                obj = [startnode]
                for k in random.sample(nodes, n):
                    obj.append(k)
                obj.append(endnode)
                path = []
                weight = 0
                lenpath = 0
                for j in range(len(obj)-1):
                    A, B = self.dijsktra(obj[j], obj[j+1], kind)
                    if A == []:
                        break
                    path.append(A)
                    weight = weight + B
                if weight < weight_final:
                    weight_final = weight
                    final_path = path    
        else:
            permuTuples = list(itertools.permutations(nodes))
            final_path = []
            weight_final = sys.float_info.max 
            for i in permuTuples:
                obj = [startnode]
                for k in i:
                    obj.append(k)
                obj.append(endnode)
                path = []
                weight = 0
                lenpath = 0
                for j in range(len(obj)-1):
                    A, B = self.dijsktra(obj[j], obj[j+1], kind)
                    if A == []:
                        break
                    path.append(A)
                    weight = weight + B
                if weight < weight_final:
                    weight_final = weight
                    final_path = path
        to_print = []
        #print(final_path)
        if final_path == []:
            return ("No path possible")
        for i in final_path:
               for j in i:
                    to_print.append(j)
        return to_print    
        
    def functionality4(self):
        H_n = int(input('Write the number associated to the starting node.\n'))
        startnodes = [int(x) for x in input('Write the nodes, each one spaced from the other, remember the last one will be always our end point.\n').split()]
        endnode = startnodes[-1]
        startnodes = startnodes[:-1]
        measure = int(input("We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. \n "))
        if measure == 1:
            return (self.dijsktrafor4( H_n, startnodes, endnode, self.adj_list_d), H_n, startnodes, measure)
        elif measure == 2:
            return (self.dijsktrafor4( H_n, startnodes, endnode, self.adj_list_t), H_n, startnodes, measure)
        elif measure == 3 :
            return (self.dijsktrafor4( H_n, startnodes, endnode, self.adj_list_n), H_n, startnodes, measure)
        
    def visualization1(self):
        plt.figure()
        plt.figure(figsize=(30,30)) 
        I = nx.Graph()
        f_1, m = self.functionality1()
        for ele in list(f_1):
            I.add_node(ele)
            I.add_edge(list(f_1)[0], ele)
        pos = {}
        labels = {}
        for node in I.nodes():
            pos[node] = ( self.coordinates[node][0] , self.coordinates[node][1])
            labels[node] = str(node)
        source = list(f_1)[0]
        #nx.draw(I, pos=pos)
        nx.draw_networkx_labels(G, pos, labels, font_size= 16 , alpha = 0.5)
        nx.draw_networkx_nodes(I, pos, nodelist=[source], node_color="crimson", node_size = 500)
        nx.draw_networkx_nodes(I, pos, nodelist=list(set(I.nodes()).difference({source})), node_color="moccasin")
        if m == 1:
            nx.draw_networkx_edges(I, pos, edge_color = "olive", width  = 2)
        elif m == 2:
            nx.draw_networkx_edges(I, pos, edge_color = "mediumaquamarine", width =2)
        elif m == 3:
            nx.draw_networkx_edges(I, pos, edge_color = "lightslategrey", width  = 2)
        plt.show()
        
    def visualization2(self):
        I = nx.Graph()
        f_2, start, m = self.functionality2()
        for ele in f_2:
            I.add_node(ele[0])
            I.add_node(ele[1])
            I.add_edge(ele[0], ele[1])
        labels = {}
        pos = {}
        for node in I.nodes():
            pos[node] = ( self.coordinates[node][0] , self.coordinates[node][1])
            labels[node] = str(node)
        plt.figure()
        plt.figure(figsize=(12,15)) 
        nx.draw_networkx_labels(G, pos, labels = {n:lab for n,lab in labels.items() if n in start}, 
                            font_size= 16 , alpha = 0.8)
        nx.draw_networkx_nodes(I, pos, nodelist=list(set(I.nodes()).difference(set(start))), node_color="lavender", node_size = 5)
        nx.draw_networkx_nodes(I, pos, nodelist=start, node_color="moccasin", node_size = 400)
        if m == 1:
            nx.draw_networkx_edges(I, pos, edge_color = "olive", width  = 2)
        elif m == 2:
            nx.draw_networkx_edges(I, pos, edge_color = "mediumaquamarine", width  = 2)
        elif m == 3:
            nx.draw_networkx_edges(I, pos, edge_color = "lightslategrey", width  = 2)
        plt.show()
        
    def visualization3(self):
        I = nx.Graph()
        f_3, start_nodes, nodes, m = self.functionality3()
        for ele in f_3:
            I.add_node(ele[0])
            I.add_node(ele[1])
            I.add_edge(ele[0], ele[1])
        labels = {}
        pos = {}
        for node in I.nodes():
            pos[node] = ( self.coordinates[node][0] , self.coordinates[node][1])
            labels[node] = str(node)
        plt.figure()
        plt.figure(figsize=(30,30)) 
        al = nodes
        al.append(start_nodes)
        nx.draw_networkx_labels(G, pos, labels = {n:lab for n,lab in labels.items() if n in al}, 
                                font_size= 16 , alpha = 0.8)
        nx.draw_networkx_nodes(I, pos, nodelist=list(set(I.nodes()).difference(set(al))), node_color="lavender", node_size = 5)
        nx.draw_networkx_nodes(I, pos, nodelist=nodes, node_color="moccasin", node_size = 400)
        nx.draw_networkx_nodes(I, pos, nodelist=[start_nodes], node_color="crimson", node_size = 500)
        if m == 1:
            nx.draw_networkx_edges(I, pos, edge_color = "olive", width  = 2)
        elif m == 2:
            nx.draw_networkx_edges(I, pos, edge_color = "mediumaquamarine", width  = 2)
        elif m == 3:
            nx.draw_networkx_edges(I, pos, edge_color = "lightslategrey", width  = 2)
        plt.show()
        
    def visualisation4(self):
        I = nx.Graph()
        f_4, start_nodes, nodes, m = self.functionality4()
        for ele in f_4:
            I.add_node(ele[0])
            I.add_node(ele[1])
            I.add_edge(ele[0], ele[1])
        labels = {}
        pos = {}
        for node in I.nodes():
            pos[node] = ( self.coordinates[node][0] , self.coordinates[node][1])
            labels[node] = str(node)
        plt.figure()
        plt.figure(figsize=(30,30)) 
        al = nodes
        al.append(start_nodes)
        nx.draw_networkx_labels(G, pos, labels = {n:lab for n,lab in labels.items() if n in al}, 
                                    font_size= 16 , alpha = 0.8)
        nx.draw_networkx_nodes(I, pos, nodelist=list(set(I.nodes()).difference(set(al))), node_color="lavender", node_size = 5)
        nx.draw_networkx_nodes(I, pos, nodelist=nodes, node_color="moccasin", node_size = 400)
        nx.draw_networkx_nodes(I, pos, nodelist=[start_nodes], node_color="crimson", node_size = 500)
        if m == 1:  
            nx.draw_networkx_edges(I, pos, edge_color = "olive", width  = 2)
        elif m == 2:
            nx.draw_networkx_edges(I, pos, edge_color = "mediumaquamarine", width  = 2)
        elif m == 3:
            nx.draw_networkx_edges(I, pos, edge_color = "lightslategrey", width  = 2)
        plt.show()    

# Read the Data
#### Previewing the data, we notice that the first 7 rows are an introduction and an a prologue for the data that follows, so we skip these

#### Let's create a Graph out of our Data

In [137]:
G = Graph()

In [138]:
with open(r"USA-road-d.CAL.gr", encoding='utf-8') as file:
    n = 0
    for line in file:
        if n > 6 :
            ww = line.split()
            G.add(ww[1], ww[2])
            G.distance(ww[1], ww[2], ww[3])
        n += 1

In [139]:
with open(r"USA-road-t.CAL.gr", encoding='utf-8') as file:
    n = 0
    for line in file:
        if n > 6 :
            ww = line.split()
            G.add(ww[1], ww[2])
            G.time(ww[1], ww[2], ww[3])
        n += 1

In [140]:
with open(r"USA-road-d.CAL.co", encoding='utf-8') as file:
    n = 0
    for line in file:
        if n > 6 :
            ww = line.split()
            G.coordinate(ww[1], ww[2], ww[3])
        n += 1

In [142]:
G.functionality1()

Give me the node as a number: 
 1
We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. 
 2
Put a number as threshold distance: 
 45325


[1,
 1048577,
 1803,
 1766,
 1767,
 1802,
 2589,
 1049992,
 1049993,
 1785,
 1050020,
 1050022,
 4,
 1050007,
 2588,
 1804,
 1050021,
 1799,
 1808,
 1805,
 1050654,
 2590,
 1050018,
 1050024,
 1050017,
 1810,
 1798,
 1800,
 1807,
 1050026,
 1050027,
 1708,
 1050019,
 1806,
 1050023,
 1050028,
 1823,
 1812,
 1811,
 1050029,
 451,
 1050038,
 1048937,
 1050870,
 1707,
 2865]

In [143]:
G.functionality2()

Write the nodes, each one spaced from the other.1 2 3 4
We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. 
 1


([(1637, 1049886),
  (1050045, 1049938),
  (1593, 1553),
  (1049947, 1714),
  (1049847, 1049846),
  (1700, 1681),
  (1050173, 1973),
  (1049863, 1618),
  (1049884, 1049864),
  (1049931, 1690),
  (1049993, 4),
  (1048577, 1767),
  (1633, 1049884),
  (1974, 1050150),
  (1050150, 1964),
  (1050159, 1969),
  (1690, 1049947),
  (1048579, 3),
  (1049923, 1676),
  (1715, 1050045),
  (1049903, 1049847),
  (2033, 1991),
  (1049938, 1700),
  (1050207, 2033),
  (1048578, 2),
  (1, 1048577),
  (1767, 1049993),
  (1049906, 1656),
  (1553, 1049821),
  (3, 1049931),
  (1656, 1049903),
  (1667, 1049906),
  (1681, 1049923),
  (1049864, 1049863),
  (1049950, 1715),
  (1049854, 1593),
  (1554, 1049822),
  (1969, 1968),
  (1991, 1048578),
  (1618, 1049854),
  (1672, 1667),
  (1049886, 1633),
  (1049821, 1554),
  (1973, 1050159),
  (1049846, 1637),
  (1049822, 1050173),
  (1968, 1974),
  (1964, 1050207),
  (4, 1048579),
  (1714, 1049950),
  (1676, 1672)],
 [1, 2, 3, 4])

In [144]:
G.functionality3()

Write the number associated to the starting node.
1
Write the nodes, each one spaced from the other.
2 3 4 5
We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. 
 3


[(1, 1803),
 (1803, 1802),
 (1802, 1050022),
 (1050022, 1799),
 (1799, 1050018),
 (1050018, 1050017),
 (1050017, 1798),
 (1798, 1708),
 (1708, 451),
 (451, 1048937),
 (1048937, 1707),
 (1707, 1705),
 (1705, 1050005),
 (1050005, 1778),
 (1778, 1050000),
 (1050000, 1774),
 (1774, 1049998),
 (1049998, 1773),
 (1773, 1777),
 (1777, 1748),
 (1748, 1049977),
 (1049977, 1749),
 (1749, 1049913),
 (1049913, 1049911),
 (1049911, 1663),
 (1663, 1825),
 (1825, 1675),
 (1675, 1049920),
 (1049920, 1049860),
 (1049860, 1552),
 (1552, 1049820),
 (1049820, 1553),
 (1553, 1049821),
 (1049821, 1554),
 (1554, 1049822),
 (1049822, 1050173),
 (1050173, 1973),
 (1973, 1050159),
 (1050159, 1969),
 (1969, 1968),
 (1968, 1974),
 (1974, 1050150),
 (1050150, 1964),
 (1964, 1050207),
 (1050207, 2033),
 (2033, 1991),
 (1991, 1048578),
 (1048578, 2),
 (2, 1048578),
 (1048578, 1991),
 (1991, 2033),
 (2033, 1050207),
 (1050207, 1964),
 (1964, 1050150),
 (1050150, 1974),
 (1974, 1968),
 (1968, 1969),
 (1969, 1050159),


In [145]:
G.functionality4()

Write the number associated to the starting node.
1
Write the nodes, each one spaced from the other, remember the last one will be always our end point.
2 3 4 5
We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. 
 1


[(1, 1048577),
 (1048577, 1766),
 (1766, 1048579),
 (1048579, 3),
 (3, 1049931),
 (1049931, 1690),
 (1690, 1049947),
 (1049947, 1714),
 (1714, 1049950),
 (1049950, 1715),
 (1715, 1050045),
 (1050045, 1049938),
 (1049938, 1700),
 (1700, 1681),
 (1681, 1049923),
 (1049923, 1676),
 (1676, 1672),
 (1672, 1667),
 (1667, 1049906),
 (1049906, 1656),
 (1656, 1049903),
 (1049903, 1049847),
 (1049847, 1049846),
 (1049846, 1637),
 (1637, 1049886),
 (1049886, 1633),
 (1633, 1049884),
 (1049884, 1049864),
 (1049864, 1049863),
 (1049863, 1618),
 (1618, 1049854),
 (1049854, 1593),
 (1593, 1553),
 (1553, 1049821),
 (1049821, 1554),
 (1554, 1049822),
 (1049822, 1050173),
 (1050173, 1973),
 (1973, 1050159),
 (1050159, 1969),
 (1969, 1968),
 (1968, 1974),
 (1974, 1050150),
 (1050150, 1964),
 (1964, 1050207),
 (1050207, 2033),
 (2033, 1991),
 (1991, 1048578),
 (1048578, 2),
 (2, 1048578),
 (1048578, 1991),
 (1991, 2033),
 (2033, 1050207),
 (1050207, 1964),
 (1964, 1050150),
 (1050150, 1974),
 (1974, 1968)

In [11]:
G.functionality1()

Give me the node as a number: 
 11
We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. 
 2
Put a number as threshold distance: 
 9000


[11, 1048584]

In [12]:
G.functionality1()

Give me the node as a number: 
 11
We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. 
 3
Put a number as threshold distance: 
 9


[11,
 1048584,
 10,
 1048583,
 2257,
 9,
 2645,
 2227,
 2646,
 2226,
 2644,
 1050360,
 2253,
 1050380]

In [9]:
G.functionality2()

Write the nodes, each one spaced from the other.33 11 44 55 22
We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. 
 1


[(11, 1048584),
 (1048584, 10),
 (10, 2257),
 (2257, 2645),
 (2645, 2227),
 (2227, 2226),
 (2226, 1050360),
 (1050360, 2253),
 (2253, 1050380),
 (1050380, 1050378),
 (1050378, 2223),
 (2223, 2224),
 (2224, 2220),
 (2220, 1050345),
 (1050345, 2205),
 (2205, 1050348),
 (1050348, 2214),
 (2214, 1050379),
 (1050379, 2251),
 (2251, 1050363),
 (1050363, 2229),
 (2229, 2237),
 (2237, 1050369),
 (1050369, 2238),
 (2238, 1050368),
 (1050368, 6),
 (6, 5),
 (5, 1048581),
 (1048581, 8),
 (8, 1050376),
 (1050376, 2246),
 (2246, 1050304),
 (1050304, 1050305),
 (1050305, 2647),
 (2647, 2157),
 (2157, 1050306),
 (1050306, 1050308),
 (1050308, 2159),
 (2159, 34),
 (34, 1048603),
 (1048603, 1048598),
 (1048598, 33),
 (33, 1048598),
 (1048598, 52),
 (52, 1050719),
 (1050719, 51),
 (51, 1048617),
 (1048617, 2472),
 (2472, 1048614),
 (1048614, 47),
 (47, 46),
 (46, 69),
 (69, 1048632),
 (1048632, 1048635),
 (1048635, 1048633),
 (1048633, 1048634),
 (1048634, 71),
 (71, 1048650),
 (1048650, 1048613),
 (1048

In [61]:
G.functionality4()

Write the number associated to the starting node.
1
Write the nodes, each one spaced from the other, remember the last one will be always our end point.
2 3 4 5 6 7 8 9
We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. 
 1
Because the nodes are more than 9 we will apply an euristic method and return a probable shortes route



'No path possible'

In [1]:
G.functionality1()

NameError: name 'G' is not defined

In [14]:
G.functionality3()

Write the number associated to the starting node.
33
Write the nodes, each one spaced from the other.
11 44 55 22
We need a function to claculate the distance, if you want metric distance digit 1, time distance 2, network distance 3. 
 1


[(33, 1048598),
 (1048598, 1048603),
 (1048603, 34),
 (34, 2159),
 (2159, 1050308),
 (1050308, 1050306),
 (1050306, 2157),
 (2157, 2647),
 (2647, 1050305),
 (1050305, 1050304),
 (1050304, 2246),
 (2246, 1050376),
 (1050376, 8),
 (8, 1048581),
 (1048581, 5),
 (5, 6),
 (6, 1050368),
 (1050368, 2238),
 (2238, 1050369),
 (1050369, 2237),
 (2237, 2229),
 (2229, 1050363),
 (1050363, 2251),
 (2251, 1050379),
 (1050379, 2214),
 (2214, 1050348),
 (1050348, 2205),
 (2205, 1050345),
 (1050345, 2220),
 (2220, 2224),
 (2224, 2223),
 (2223, 1050378),
 (1050378, 1050380),
 (1050380, 2253),
 (2253, 1050360),
 (1050360, 2226),
 (2226, 2227),
 (2227, 2645),
 (2645, 2257),
 (2257, 10),
 (10, 1048584),
 (1048584, 11),
 (11, 1048584),
 (1048584, 10),
 (10, 2257),
 (2257, 2645),
 (2645, 2227),
 (2227, 2226),
 (2226, 1050360),
 (1050360, 2253),
 (2253, 1050380),
 (1050380, 1050378),
 (1050378, 2223),
 (2223, 2224),
 (2224, 2220),
 (2220, 1050345),
 (1050345, 2205),
 (2205, 1050348),
 (1050348, 2214),
 (2214,

In [ ]:
G.visualization1()

In [ ]:
G.visualization2()

In [ ]:
G.visualization3()

In [ ]:
G.visualization4()